In [5]:
from neo4j import GraphDatabase

url = 'neo4j://localhost:7687'
user = 'neo4j'
password = 'securepassword'

driver = GraphDatabase.driver(url, auth=(user, password))


1. Найдём все фильмы, в которых играл Tom Hanks

In [ ]:
def movies_tom_hanks(tx):
    result = tx.run(
        "MATCH (person: Person{name:'Tom Hanks'})-[:ACTED_IN]->(movie:Movie) RETURN movie.title AS movie_title"
    )
    return [record for record in result]

In [16]:
with driver.session() as session:
    movie = session.execute_read(movies_tom_hanks)
    
    for p in movie:
        print(f'{p["movie.title"]}')

Joe Versus the Volcano
The Da Vinci Code
A League of Their Own
Apollo 13
Charlie Wilson's War
The Green Mile
Cast Away
That Thing You Do
The Polar Express
Sleepless in Seattle
Cloud Atlas
You've Got Mail
The Da Vinci Code
Cloud Atlas
Charlie Wilson's War
Cast Away
You've Got Mail
Apollo 13
The Polar Express
The Green Mile
Sleepless in Seattle
A League of Their Own
That Thing You Do
Joe Versus the Volcano
Charlie Wilson's War
The Green Mile
Cast Away
That Thing You Do
You've Got Mail
Cloud Atlas
A League of Their Own
The Polar Express
Apollo 13
The Da Vinci Code
Joe Versus the Volcano
Sleepless in Seattle


2. Найти режиссёров, которые сняли больше одного фильма

In [37]:
def producer_no_zero(tx):
    result = tx.run(
        """
        MATCH (producer:Person)-[:PRODUCED]->(movie:Movie)
        WITH producer, count(movie) as count_movie
        WHERE count_movie > 1
        RETURN producer, count_movie 
"""
    )
    return list(result)

In [38]:
with driver.session() as session:
    producer = session.execute_read(producer_no_zero)
    
    for p in producer:
        print(f'{p["producer"].get("name")}')

Joel Silver
Lilly Wachowski
Lana Wachowski
Joel Silver
Lilly Wachowski
Lana Wachowski
Joel Silver
Lilly Wachowski
Lana Wachowski


3. Средний год выпуска фильмов, в которых играл конкретный актёр

In [44]:
def avg_films(tx, name):
    result = tx.run(
        """
        MATCH (person: Person{name:$name})-[:ACTED_IN]->(movie:Movie)
        RETURN AVG(movie.released) as avg_year
""",
name=name
    )
    record = result.single()
    
    return record["avg_year"] if record else None

In [48]:
with driver.session() as session:
    avg_year_film = session.execute_read(avg_films, "Ethan Hawke")
    print(avg_year_film)

1999.0
